**Pytorch Tutorial**


In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import time

import torch
import torchvision
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

## Tensors

Tensors are the base data structures of PyTorch which are used for building different types of neural networks. They can be considered as the generalization of arrays and matrices; in other words, tensors are N-dimensional matrices or multidimensional arrays.

PyTorch tensors are similar to NumPy’s n-dimensional arrays. But unlike ndarrays of numpy, these tensors can be stored on a GPU RAM as well as the CPU RAM, and than to be multiplied efficientlly on the small many cores of the GPU (this is not the case with NumPy arrays). This is a major advantage of using tensors.

###  Tensor Initialization

Tensors can be created by using a list:

In [ ]:
x = torch.tensor([1,2,3])
print(x)

You can specify the data type of the tensor by using the ``dtype`` argument,
similarily to numpy, you can check the type of the tensor by using the ``dtype`` attribute:

In [ ]:
x = torch.tensor([1,2,3], dtype = torch.float32)
print(x)
print(x.dtype)

It's important to note here that one way to evaluate a GPU is by its TFLOPS - the capability of a processor to calculate one trillion floating-point operations per second. These operations rate usually refer to a 32bit floating point and abiously depends on the tensot type. operations on a 64bit floating point will be half the rate and so on.

Let’s say we want a matrix of shape 3*3 having all zeros/ones. Take a moment to think – how can we do that using NumPy?

In [ ]:
a = np.zeros((3,3))
b = np.ones((3,3))

print(a)
print(b)

Similar to NumPy, PyTorch also has the zeros() and ones() function which takes the shape as input and returns a matrix of zeros or ones , respectively, of a specified shape.

In [ ]:
a = torch.zeros((3,3))
b = torch.ones((3,3))

print(a)
print(b)

The ``random.randn()`` function returns random numbers that follow a standard normal distribution. 

We can initialize a similar matrix of random numbers using PyTorch:



In [ ]:
a = torch.randn(2,3)

print(a)

### Tensor Copying

As you will see later in this chapter,
most of the tensor operations create a view of the origin tensor.

Since it is a room for errors you must notice when you need to create a copy of your origin tensor.

Unlike numpy, in torch the syntax is a bit different.

The function `copy_` copies the tensor given as a parameter.

In [ ]:
a = torch.randn(3,3)
print(a, "\n")

b = torch.empty(3,3)
b.copy_(a)
print(b, "\n")

a[0,0] = 10
print(a[0,0], "\n")
print(b[0,0])

The function `clone` also copies a tensor.

However, unlike `copy_`, this function is recorded in the computation graph. 
(As you will see later in this tutorial, it means that gradients propagating to the cloned tensor will propagate to the original tensor.)

In [ ]:
a = torch.randn(3,3)
print(a, "\n")

b = torch.empty(3,3)
b = a.clone()
print(b, "\n")

a[0,0] = 10
print(a[0,0], "\n")
print(b[0,0])

### Shaping and Reshaping

You can get the shape of a tensor using the ``.size()`` method or just the `.shape` attribute.

In [ ]:
a = torch.randn(2,3)

print(a.size())
print(a.shape)

We can use the `.view()` function and pass the required shape as a parameter. 

It will return a tensor with the new shape. 

Pay attention that the returned tensor isn't a copy of the original tensor, it will share the underling data with the original tensor.

Let’s try to convert the above tensor of shape (2,3) to a tensor of shape (6,1):



In [ ]:
b = a.view(6,1)
print(b)
print(b.shape)

In [ ]:
a[0][0]=5
print(b)

If you know you want, for example, 6 rows however, you want torch to conclude the number of columns, you should send -1 as the columns dim argument

In [ ]:
a = torch.randn(3,4,5)
b = a.view(-1,12)
print(b)
print(b.shape)

If you have two twnsors with the same amount of elements and you want to make their shape the same you can pass one's shape as a parameter to the `view` function

In [ ]:
a = torch.randn(3,4,5)
print(a.size())
b = torch.randn(6, 10)
print(b.size(), "\n")

b = b.view(a.size())
print(b.size())

However, you can do it more elegantlly using the `view_as` function:

In [ ]:
a = torch.randn(3,4,5)
print(a.size())
b = torch.randn(6, 10)
print(b.size(), "\n")

b = b.view_as(a)
print(b.size())

You can also use the `torch.reshape` method,
however, pay attentaion that torch.reshape may return a copy or a view of the original tensor - `torch.reshape` method will try to share the memory of the returned tensor, but if that complicates things for the torch architecture torch will make a clone of it and will return a new reshaped tensor. Thus you can not count on that to return a view or a copy.

In order to transpose a 2D tensor, you can use ``torch.t`` as follows:

In [ ]:
a = torch.randn(2,3)

print(a)
print(f"tensor a shape is {a.shape} \n")

a_t = torch.t(a)
print(a_t)
print(f"the transpose of tensor a shape is {a_t.shape} \n")

Pay attention that 0-D and 1-D tensors are returned as is. 

In [ ]:
a = torch.randn(5)
print(a.shape, "\n")

a_t = torch.t(a)
print(a_t.shape)

If you want to transpose a tensor with more than 2 dimensions, you can use the `permute` function:

In [ ]:
a = torch.randn(3, 4, 5)
print(a.size())
b.copy_(a.permute(2, 0, 1))
print(b.size())

print(a[1,2,3] == b[3,1,2])
print(a[1,2,3] == b[1,2,3])

### Mathematical Operations

Let’s now see how we can do mathematical operations using PyTorch on tensors. So, first, let’s initialize two tensors:

In [ ]:
a = torch.randn(2,3)
b = torch.randn(2,3)

print(a)
print(b)

There are 3 different ways to perform mathematical operations:

1. Well known  mathematical operators:

In [ ]:
# addition
print(a+b, "\n")       # equivalent to: torch.add(a, b)

# subtraction
print(b-a, "\n")       # equivalent to: torch.sub(b, a)

# elementwise multiplication
print(a*b, "\n")       # equivalent to: torch.mul(a, b)

# dot product
print(a@torch.t(b), "\n")       # equivalent to: torch.dot(a, b)

# division
print(b/a, "\n")       # equivalent to: torch.div(b, a)


2. Torch methods

In [ ]:
# addition
print(torch.add(a, b), "\n")

# subtraction
print(torch.sub(b, a), "\n")

# elementwise multiplication
print(torch.mul(a, b), "\n")

# dot product
print(torch.mm(a, torch.t(b)), "\n")

# division
print(torch.div(b, a), "\n")


3. Torch inplace methods

  The inplace methods functionality is the same as the regular methods shown above, accept that their reult is written to the object they were operated on.
The inplace methods always followed by an underscore.

In [ ]:
# addition
a.add_(b)
print(a, "\n")


### Concatenating

Let’s say we have two tensors as shown below and we want to concatenate them


In [ ]:
a = torch.tensor([[1,2],[3,4]])
b = torch.tensor([[5,6],[7,8]])

print(a, "\n")
print(b)

In [ ]:
# concatenating vertically
torch.cat((a,b))

As you can see, the second tensor has been stacked below the first tensor. We can concatenate the tensors horizontally as well by setting the dim parameter to 1:


In [ ]:
# concatenating horizontally
torch.cat((a,b),dim=1)

### Accumulating Functions

The accumulating functions are mostly the same in torch tensors as in numpy arrays.


In [ ]:
a = torch.tensor([[1,2],[3,4]], dtype=torch.float32)
print(a)

In [ ]:
a_sum = a.sum()
print(f"a.sum() = {a_sum}")

Similarily to numpy, you can specify the axis which you want to accumulate along: 

``axis=0`` for columns and ``axis=1`` for rows (as oppose to numpy).

In [ ]:
# the mean of each column in the tensor a:
a.mean(axis=0)

A very important function to use after you have done an accumulationg operation is the ``torch.item`` function.
This function return a scalar from a tensor which contain only one item in it.

In [ ]:
x = torch.tensor([[1]])
print(x, "\n")

print(x.item())

Unlike numpy, `torch.max` return both the maximal element in  a tensor and its index.

Use `dim=1` to maximize over each row of the tensor columns and `dims=0` to maximize over each column of the tensor rows.

Take a moment and think, why is it useful? Yoy will see in the upcoming chapters.

In [ ]:
x = torch.randn(3,5)
print(x, "\n")

value, index = torch.max(x, dim=1) 
print(value)
print(index)

### Numpy Bridge

Converting a torch Tensor to a numpy array and vice versa is a breeze. The torch Tensor and numpy array will share their underlying memory locations, and changing one will change the other.

In [ ]:
a = np.array([[1,2],[3,4]])
print(a, "\n")

tensor = torch.from_numpy(a)
print(tensor)

Pay attenation that the numpy array and the torch tensor share the same memory,
if you change the numpy array you chang the tensor and vice versa.

In [ ]:
a[0,0] = 5
print(tensor, "\n")

tensor[0,0] = 6
print(a)

In [ ]:
b = tensor.numpy()
print(b)

## Autograd: Automatic Differentiation

Central to all neural networks in PyTorch is the ``autograd`` package. Let’s first briefly visit this.

The autograd package provides automatic differentiation for all operations on Tensors. 

It is a define-by-run framework, which means that your backprop is defined by how your code is run, and that every single iteration can be different.

``torch.Tensor``, which you saw above, is the central class of the package. 

If you set its attribute ``.requires_grad`` as ``True``, it starts to track all operations on it. 

When you finish your computation you can call ``.backward()`` and have all the gradients computed automatically. 

The gradient for this tensor will be accumulated into .grad attribute.

There’s one more class which is very important for the autograd implementation - a ``Function``.

Tensor and Function are interconnected and build up an acyclic graph, that encodes a complete history of computation. Each tensor has a ``.grad_fn`` attribute that references to a Function that has created the Tensor (except for Tensors created by the user - their ``grad_fn`` is ``None``).

If you want to compute the derivatives, you can call ``.backward()`` on a Tensor. If Tensor is a scalar (i.e. it holds a one element data), you don’t need to specify any arguments to ``backward()``, however if it has more elements, you need to specify a gradient argument that is a tensor of matching shape.

In [ ]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

In [ ]:
y = x + 2
print(y, "\n")

z = y * y * 3
print(z, "\n")

out = z.mean()
print(out)

Let’s backprop now. 

In [ ]:
out.backward()

Print gradients d(out)/dx

In [ ]:
print(x.grad)

Calculating the gradients with respect to all of our parameters is time-consuming.

Therefore, there are times when we don't want the gradients to be computed.

There are 3 ways to stop the gradients calculations:

1. Updating the ``.requires_grad`` attribute as ``False``

In [ ]:
a = torch.randn(3, requires_grad=True)
print(a)
print(a.requires_grad, "\n")

a.requires_grad_(False)

print(a)
print(a.requires_grad)

2. Call .detach()

In [ ]:
a = torch.randn(3, requires_grad=True)
print(a)
print(a.requires_grad, "\n")

b = a.detach()

print(b)
print(b.requires_grad)

Pay attention that the gradients above share the same memory.

3. Using the ``with`` statement

In [ ]:
a = torch.randn(3, requires_grad=True)
print(a)
print(a.requires_grad, "\n")

with torch.no_grad():
  b = a+2
  print(b)
  print(b.requires_grad)

As mentioned earlier, the gradients are accumulated in the ``.grad``  attribute.

Let's see an example:

In [ ]:
weights = torch.ones(4, requires_grad=True)

for epoch in range(3):
    model_output = (weights*3).sum()
    model_output.backward()
    
    print(weights.grad)


That isn't what we want.

We must zero the gradients before each iteration.

In [ ]:
weights = torch.ones(4, requires_grad=True)

for epoch in range(3):
    model_output = (weights*3).sum()
    model_output.backward()
  
    print(weights.grad)

    weights.grad.zero_()

##  Training Pipeline Overview: Model, Loss, and Optimizer

There are 3 common steps in order to make NN using pytorch.
1. Design model (input, output, forward pass with different layers).
2. Construct loss and optimizer.
3. Training loop:
      - Forward - compute prediction and loss.
      - Backward - compute gradients.
      - Update weights.

Its important to understand the different components of the process and the well defined separation between them.
- Forward - only needs the X features, and the current weights.
- Backward - only needs the y label and the y_predicted to be given to the loss. So the gradients can be acumulated with respect to the loss (being calculated using ``loss.backprop()``).
- Update weights - only needs the gradients and the optimizing method.

##  Linear Regression

Here is our first very simple example of NN.

This NN implements the simple linear regression.

The most important thing to take from this example is the structure of our code.

First, let's make a small dataset:

In [ ]:
X_numpy, y_numpy = datasets.make_regression(n_samples=100, 
                                            n_features=1, 
                                            noise=15, 
                                            random_state=42)

X = torch.from_numpy(X_numpy.astype(np.float32))
y = torch.from_numpy(y_numpy.astype(np.float32))
y = y.view(y.shape[0], 1)

n_samples, n_features = X.shape

In [ ]:
plt.scatter(X , y)
plt.title("Linear Regression Data")
plt.xlabel("X")
plt.ylabel("y")

Step No. 1:

We define our NN as a class that inherite from ``nn.Module``.

In this class we must implement two methods: the ``__init__`` method
and the ``forward`` method.

In the ``__init__`` method we firstly calling ``nn.Module`` ``__init__`` and then we define our NN layers, their input and outputs sizes.

The  ``forward`` method performs the forward propagtion using the layers defined in the initialization. Most layers are already implemented for you in the ``nn`` module of pytorch. the first layer to be used is a ``nn.Linear`` layer, defined by its input dimenstions and output dimensions, storing the wights inside. In order to apply the layer one need just to pass an X to it as if it was a function.

In [ ]:
class LinearRegression(nn.Module):

  def __init__(self, input_dim, output_dim):
    super(LinearRegression, self).__init__()

    self.lin = nn.Linear(input_dim, output_dim)

  def forward(self, x):
    return self.lin(x)   

In [ ]:
model = LinearRegression(n_features, 1)
model

Model class was made so it will contain layers that operate in a sequence. As such, a very convineint way to reffer to all parameters of all layers is the method ``.parameters()``.

In [ ]:
w, b = model.parameters()

In [ ]:
plt.plot(X, w.item()*X+b.item(), c="r")
plt.scatter(X, y)
plt.title("Initial Model")
plt.xlabel("x")
plt.ylabel("y")

Step No 2.

We define our loss function from ``torch.nn`` and our optimizer from ``torch.optim``. 

Notice that as ``nn.Linear``, a loss function is just another layer function operating on a layer output - simply with no weights stored inside. which puts it in the ``torch.nn`` module.

Also notice how an optimizer needs to accept the parameters on which to optimize on. In the example above we chose to include all parameters off the model in the optimizing process. In other scenarios, For example in transfer learning, we would want to limit the parameters to be optimizing on just the last layers and not change the first layers.

In [ ]:
learning_rate = 0.01
n_iters = 100

loss = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

Step No 3.

Here we perform our training loop.

- forward propagation.
- computing loss.
- back propagation.
- parameters update.

In [ ]:
for epoch in range(n_iters):
    # predict = forward pass with our model
    y_predicted = model(X)

    # loss
    l = loss(y_predicted, y)

    # calculate gradients = backward pass
    l.backward()

    # update weights
    optimizer.step()

    # zero the gradients after updating
    optimizer.zero_grad()

    if (epoch+1) % 10 == 0:
        w, b = model.parameters()
        print('epoch ', epoch+1,' loss = ', l.item())


In [ ]:
w, b = model.parameters()

In [ ]:
plt.plot(X, w.item()*X+b.item(), c="r")
plt.scatter(X, y)
plt.title("Final Model")
plt.xlabel("x")
plt.ylabel("y")

Try running the training again and examine what happened. Do you expect that to happen?

## ✍️  Logistic Regression

In this section, you ware going to implement logistic regression using pytorch.

A short reminder of logistic regression:

Let  $W$ be our parametrs matrix and $b$ our bias.

The logistic function is defined by the following formula of the sigmoid function:

$\sigma(x; W,b) = \frac{1}{1+e^{-(W\cdot x+b)}}$


✍️ Step No. 0:

Load the data and shape it so you can train a model on it.

In [ ]:
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

n_samples, n_features = X.shape

### ✍️ Split your data to train set and test set. Use 20% of your data as test.
### START CODE HERE ### (1 line of code)

### END CODE HERE ###


### ✍️ Scale your data using the StandardScaler.
### START CODE HERE ### (~3 lines of code)

### END CODE HERE ###

### ✍️ Transform your data and labels to tensors. 
### Don't forget to convert their dtype to float32.
### START CODE HERE ### (4 lines of code)

### END CODE HERE ###

### ✍️ Reshape your lables.
### START CODE HERE ### (2 lines of code)

### END CODE HERE ###


✍️ Step No. 1:

Define the LogisticRegression model. (read relevant layers in the nn documentaion)

In the ``__init__`` method, define the layer and the activation.

In the ``forward`` method, define forward propagartion.

Finally, initiate a new LogisticRegression instance.

In [ ]:
class LogisticRegression(nn.Module):
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        ### START CODE HERE ### (2 lines of code)

        ### END CODE HERE ###

    def forward(self, x):
        ### START CODE HERE ### (~1-3 lines of code)

        ### END CODE HERE ###

### START CODE HERE ### (1 line of code)

### END CODE HERE ###
print(model)

✍️ Step No 2.

Now, we define our loss function as the binary cross-entropy loss,

and stochastic gradient-descent as our optimization algorithm.

In [ ]:
num_epochs = 100
learning_rate = 0.01

### START CODE HERE ### (2 lines of code)

### END CODE HERE ###

✍️ Step No 3.

Now, we perform our training loop.

- forward propagation.
- computing loss.
- back propagation.
- parameters update.

In [ ]:
for epoch in range(num_epochs):

  ### Forward pass
  ### START CODE HERE ### (1 line of code)

  ### END CODE HERE ###

  ### Loss calculation
  ### START CODE HERE ### (1 line of code)

  ### END CODE HERE ###

  ### Backward pass
  ### START CODE HERE ### (3 lines of code)

  ### END CODE HERE ###

  ### Print loss every 10 epochs
  ### START CODE HERE ### (2 lines of code)

  ### END CODE HERE ###

✍️ Finally, We must check how our model perforn on the test set.

In this part we make a forward pass, however, we don't want the gradient to be calculated.

In [ ]:
### Compute the test accuracy. 
### Use 0.5 as the classification border, samples with sigmoid result greater
### or equal to 0.5 will be classified as class 1, 
### and all the others will be classifies as class 0.
### START CODE HERE ### (~5 lines of code)

### END CODE HERE ###

## Dataset and DataLoader

As You probably saw in Andrew's course,
the gradient computation is not efficient for the whole data set.

Therefore, we divide the dataset into small batches, and compute the gradient on each batch.

A short reminder about the defenitions regarding this part:
- epoch - one forward and backward pass of ALL training samples.
- batch_size - number of training samples used in one forward/backward pass.
- number of iterations - number of passes, each pass (forward+backward) using #batch_size number of sampes.

e.g : 100 samples, batch_size=20 -> 100/20=5 iterations for 1 epoch.





As explained above it might be a good idea to an object representing a Dataset to so we can feed the model in training time.

In order to implement such a Dataset, we will inherit the class ``Dataset`` of pytorch.

We will implement the following methods: ``__init__``, ``__getitem__`` , and ``__len__``,

using the scikit-learn wine dataset.

In [ ]:
class WineDataset(Dataset):

    def __init__(self):
        data = datasets.load_wine(return_X_y=True)
        self.n_samples = len(data[1])

        self.x_data = torch.from_numpy(data[0])
        self.y_data = torch.from_numpy(data[1]) 

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples


# create dataset
dataset = WineDataset()

Now, let's see that we can get the first sample:

In [ ]:
first_data = dataset[0]
features, labels = first_data
print(features, "\n", labels)

Once we defined our ``Dataset`` object we can use the already implemented ``DataLoader`` class wraping the ``Dataset`` object with batching skills and more.

The main aim of the ``DataLoader`` is to provide us a convinient way to iterate through the batches of our dataset.



In [ ]:
data_loader = DataLoader(dataset=dataset,
                          batch_size=5,
                          shuffle=True,
                          num_workers=2)

We have just defined our data loader,

now, let's convert it to an iterator and look at one random sample.

In [ ]:
dataiter = iter(data_loader)
data = dataiter.next()
features, labels = data
print(features, "\n", labels)

✍️ Iterate through the batches of the dataset.
For each batch print how much samples it has from each class.

In [ ]:
### START CODE HERE ### (~5 lines of code)
for i, (inputs, labels) in enumerate(data_loader):
  class_0 = sum(labels == 0).item()
  class_1 = sum(labels == 1).item()
  class_2 = sum(labels == 2).item()

  print(f'In batch {i+1}: class 0 : {class_0},  class 1: {class_1},  class 2 : {class_2}')
### END CODE HERE ###

This exercise purpose is to let you find on your own the best and most convinient way to write the ``for`` loop command that iterate througt the batches.

## Transformers

Transforms can be applied to PIL images, tensors, ndarrays, or custom data
during creation of the DataSet.

You can find the complete list of built-in transforms here: 
https://pytorch.org/docs/stable/torchvision/transforms.html

Usually, transformers are sent while initializing the dataset as an optional paramrter.
Then the transform, if it exists, is being operated in the ``__getitem__`` method. 

In this way one can use transformers to delay some pre-process to realtime or make the pre-process depend on the batch itself. 
But mainly this is used to do augmentaions that varies with every iteration, and has an intrinsic randomness. We will see some examples in this chapter.

Some common transforms which you can find implemented in ``torchvision.transforms``:


- On Images:

> CenterCrop, Grayscale, Pad, RandomAffine, RandomCrop, RandomHorizontalFlip, RandomRotation, Resize, Scale.



- On Tensors:

> LinearTransformation, Normalize, RandomErasing.

- Conversion:

> ToPILImage: from tensor or ndrarray.

> ToTensor : from numpy.ndarray or PILImage.

- Custom:

> Write your own class.

In [ ]:
class LinearData(Dataset):
  
  def __init__(self, n_samples=100, transform=None):
    X,y = datasets.make_regression(n_samples=n_samples,
                                   n_features=1,
                                   noise=15,
                                   random_state=42)   
    self.x = X
    self.y = y.reshape((n_samples, -1))
    self.n_samples = n_samples
    self.transform = transform

  def __getitem__(self, index):
    sample = self.x[index], self.y[index]
    if self.transform:
      sample = self.transform(sample)
    return sample

  def __len__(self):
    return self.n_samples


In [ ]:
class ToFloat32:
    def __call__(self, sample):
        inputs, targets = sample
        return inputs.astype("float32"), targets.astype("float32")

In [ ]:
print('Without transform:')
dataset = LinearData(n_samples=500)
features, labels = dataset[0]
print(features.dtype, labels.dtype, "\n")

print('With transform to float32:')
dataset = LinearData(n_samples=500, transform=ToFloat32())
features, labels = dataset[0]
print(features.dtype, labels.dtype)

✍️ Implement ToTensor transform.

  Show that your transform works.

In [ ]:
### START CODE HERE ### (~4 lines of code)
class ToTensor:
  def __call__(self, sample):
    inputs, targets = sample
    return torch.from_numpy(inputs), torch.from_numpy(targets)
### END CODE HERE ###  

In [ ]:
### START CODE HERE ### (~8 lines of code)
print('Without transform:')
dataset = LinearData(n_samples=500)
features, labels = dataset[0]
print(type(features), type(labels), "\n")

print('With tensor transform:')
dataset = LinearData(n_samples=500, transform=ToTensor())
features, labels = dataset[0]
print(type(features), type(labels))
### END CODE HERE ###  

✍️ Implement MulTransform. This transformer have to get a factor and multiply the labels with this given factor.

In [ ]:
### START CODE HERE ### (~7 lines of code)
class MulTransform:
    def __init__(self, factor):
        self.factor = factor

    def __call__(self, sample):
        input, target = sample
        target *= self.factor
        return input, target
### END CODE HERE ###  

✍️ Now, after we have built 3 transformers, let's compose them.

Using ``torchvision.transforms`` documentation, write code which compose the above transformers.

Your transform have to convert a numpy array to tensor of type float32 and multiply the labels of the data by 4. Show your code works. 

In [ ]:
### START CODE HERE ### (~12 lines of code)
composed = torchvision.transforms.Compose([ToFloat32(), ToTensor(), MulTransform(4)])
plt.figure()

for transform, title in zip([None, composed], ["Without transform", "With transform"]):
  dataset = LinearData(n_samples=500, transform=transform)
  xs , ys = [], []
  for i in range(len(dataset)):
    xs.append(dataset[i][0].item())
    ys.append(dataset[i][1].item())
  plt.scatter(xs, ys, label = title)
  
plt.legend()
plt.xlabel("x")
plt.ylabel("y")
plt.title("Linear dataset")
### END CODE HERE ###  

## ✍️ Feed-Forward Neural Network

In this chapter you will write your first nueral network on your own!

In this task we will use the well known MNIST dataset. Remember that this is a multiclass classification problem! 

Our nueral network will have 1 hidden layers.

At first, let's define some hyper-parameters:

In [ ]:
input_size = 784 # 28x28
hidden_size = 240 
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001

Now, we define our dataset and dataloader for both the train and test set:

In [ ]:

train_dataset = torchvision.datasets.MNIST(root='./data', 
                                           train=True, 
                                          transform=torchvision.transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data', 
                                          train=False, 
                                          transform=torchvision.transforms.ToTensor())


train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

✍️ Define your nueral network class. Use ReLU as your activation.

In [ ]:
### START CODE HERE ### (~12 lines of code)

### END CODE HERE ###  

✍️ Initialize your model with the above hyper-parameters. 

In [ ]:
### START CODE HERE ### (~1 line of code)

### END CODE HERE ###  

✍️ Define the loss (which one should you choose? check the documentation and read it deeply! Fix your model definition if needed.) and use SGD as your optimization algorithm.

In [ ]:
### START CODE HERE ### (2 lines of code)
 
### END CODE HERE ###  

✍️ Write the training loop:

In [ ]:
start = time.time()

### START CODE HERE ### (~10 lines of code)

### END CODE HERE ### 

end = time.time()

In [ ]:
print(f"Training took {end-start} seconds.")

✍️ Test your model on the test data. 

Make sure you don't waste time computing things you don't need! 

What is your accuracy?

In [ ]:
### START CODE HERE ### (~10 lines of code)

### END CODE HERE ### 

Our accuracy is much better than a random guess, however, we want to improve it!

In order to improve our model preformence we should check whether our model overfits the data or try different hyper-parmeters.

In order to check if our model overfits the data we can plot the loss of the training and the validation as function of the epoch number.
We will not get into it now, but you will do it in your next exercices!

So, now we will try different hyper-parameters in order to achieve better preformance.
However, training took too much time! We must use our resources wisely!
In the next chapter we will learn how to use the gpu!

## Using the GPU

By default all tensors are created on the CPU,
but you can also move them to the GPU (if it's available).

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

You can directly create a tensor on GPU:

In [ ]:
x = torch.rand(5,3) 
print(x, "\n")
y = torch.ones_like(x, device=device)
print(y)

Or you can just move it to the GPU. 

Additional option to indicate device is to use strings ``.to("cuda")``

In [ ]:
x = x.to(device)
print(x)                      

It's not possible to convert tensor which is on the GPU to a numpy array because numpy cannot handle GPU tenors.

In [ ]:
z = x + y
print(z)
z = z.numpy() 

You have to move the tensor to the CPU and then convert it to a numpy array.


In [ ]:
z = z.to("cpu")       
z = z.numpy()
print(z)

Now, after you have learnt how to run tensors on the GPU let's try it!

✍️ Try to run your model from the previous chapter on the GPU.
In order to make it you have to move any inputs/tensors and your model to the GPU.

Did your time performance got better?





Probably, your answer is no.

This can happen when the cost of transferring data between RAM and GPU memory is more than the speedup of parallel computation on the GPU.

It can happen when your model is quite small, or in case when you have too many transfers of data in your forward() function.

If you still want to see improvment in the time preformance, increase the number of epochs and use only 1 batch (batch = all your dataset) 

## RNNs